# Importing Libraries

In [1]:
from bs4 import BeautifulSoup as bs
import pandas as pd
import numpy as np
import requests 
import lxml
import os

# Defining URLs for Pages And Downloading Pages Using Libraries

In [2]:
url1 = 'https://stackoverflow.com/tags'

url2 = 'https://stackoverflow.com'

url = 'https://stackoverflow.com/questions/tagged/javascript'

In [3]:
response = requests.get(url1)

response

<Response [200]>

In [4]:
tag_page = response.text

# Using BeautifulSoup For Parsing the page And defining Code For extracting Data

In [5]:
soup =bs(tag_page, 'html.parser')

In [6]:
def get_toipc_titles(soup):
    topic_tags ='post-tag'
    topic = soup.find_all('a',{'class' : topic_tags})
    topic_titles = []
    for tag in topic:
        topic_titles.append(tag.text)
    return topic_titles

def get_describtion_tags(soup):
    describtion_tags ='flex--item fc-medium mb12 v-truncate4'
    describtion = soup.find_all('div',{'class' : describtion_tags})
    desc_text = [d.get_text().strip() for d in describtion]
    return desc_text

def get_topic_link(soup):
    link_tag = 'post-tag'
    link = soup.find_all('a',{'class' : link_tag})
    topic_link = []
    for tag in link:
        topic_link.append(url2 + tag['href'])
    return topic_link

def scrape_tags():
    tags_url = 'https://stackoverflow.com/tags'
    response = requests.get(tags_url)
    if response.status_code != 200:
        raise Exception('Cannot load page {}'.format(tags_url))        
    topics_df= {
        'Tag Topic' : get_toipc_titles(soup) , 
        'Describtion': get_describtion_tags(soup),
        'Topic link' : get_topic_link(soup)
    }
    return pd.DataFrame(topics_df)   

# Saving This Data Into CSV File

In [7]:
scrape_tags().to_csv('Tags Page', index=None )

# Code For Extracting Data From Tags Page

In [8]:
link_tag = 'post-tag'
link = soup.find_all('a',{'class' : link_tag})
topic_link = []
for tag in link:
    topic_link.append(url2 + tag['href'])
    
topic_link[2]

'https://stackoverflow.com/questions/tagged/java'

In [9]:
tagged_url = 'https://stackoverflow.com/questions/tagged/java'

In [10]:
def get_tagged_title(soup2):
    titles_tag = 's-post-summary--content-title'
    titles = soup2.find_all('h3', {'class' : titles_tag })
    title = [t.get_text().replace("\n","") for t in titles]
    return title
    

def get_content(soup2):
    post_content = 's-post-summary--content-excerpt'
    content = soup2.find_all('div', {'class' : post_content})
    content_text =[c.get_text().strip().replace("-", "").replace("\r", "").replace("\n", " ") for c in content]
    return content_text

def get_tagged_votes(soup2):
    votes_tag = 's-post-summary--stats-item s-post-summary--stats-item__emphasized'
    votes = soup2.find_all('div', {'class' : votes_tag})
    votes_text = [v.get_text().strip() for v in votes]
    vote = [v.replace("\n", " ").replace("-", "") for v in votes_text]
    return vote

def get_tagged_answered(soup2):
    answeres_tag = soup2.find_all('span', {'class' : 's-post-summary--stats-item-number'})
    answeres = [a.get_text().replace("-","") for a in answeres_tag]
    answere=answeres[1::3]
    return answere

def get_tagged_view(soup2):
    answeres_tag = soup2.find_all('span', {'class' : 's-post-summary--stats-item-number'})
    answeres = [a.get_text().replace("-","") for a in answeres_tag]
    view = answeres[2::3]
    return view

def scrape_tagged(tagged_url):
    response2 = requests.get(tagged_url)
    if response2.status_code != 200:
        raise Exception('Cannot load page {}'.format(tagged_url)) 
    soup2 = bs(response2.text, 'html.parser')
    tagged_df= {
        'Title' : get_tagged_title(soup2) , 
        'Post Of Tag': get_content(soup2),
        'Votes On Post' : get_tagged_votes(soup2),
        'Answeres On Post': get_tagged_answered(soup2),
        'Views On Post' : get_tagged_view(soup2)
    }
    return pd.DataFrame(tagged_df)   

In [11]:
scrape_tagged(tagged_url).to_csv('Java Tagged Page', index=None )

In [12]:
url3 = topic_link[3]

In [13]:
scrape_tagged(topic_link[4]).to_csv('C# Tagged Page', index=None )